# Задание
___  
Для обучения моделей требуется большое количество данных.  
Для разметки этих данных часто пользуются услугами краудсорсинга.  
Суть краудсорсинга состоит в том, что много людей делают простые задания и таким образом размечают данные.  
Эти люди могут быть не вполне квалифицированными, в связи с чем могут довольно часто ошибаться, поэтому не всем их ответам можно доверять.  
**Вы будете аналитиком**, которому нужно оценить результаты разметки данных в краудсорсинге.  
Для этого у вас будет датасет со всеми ответами исполнителей заданий на задания. Датасет в формате csv находится здесь:   https://disk.yandex.ru/d/POnzuqYYF76uuw.  
В датасете есть следующая информация:  
- worker - это уникальный айди исполнителя заданий,  
- task - это уникальный айди задания,  
- label - это ответ исполнителя в данном задании.  
  
Чтобы проверить точность исполнителей в разметку мы добавили задания с известным нам правильным ответом и дали исполнителям ответить на них (исполнители не знают правильный ответ).  Ответы на эти задания лежат в файле в json-формате по ссылке:  https://disk.yandex.ru/d/9ALcR4nh0sqo4g.

**Что нужно сделать:**  
* нужно получить ответы на все задания по методу большинства- то есть окончательным ответом будет тот, за который проголосовало больше исполнителей.  
* Далее нужно объединить датасеты и оценить точность каждого исполнителя.  
* После этого по желанию можно предложить метод агрегации ответов исполнителей с учетом.  
  
Чтобы было проще работать над тестовым я напишу алгоритм, по которому буду проверять:  

1. Датасеты загружены и по ним получена основная информация
2. По задачам в датасете получены ответы с помощью метода мнения большинства
3. Датасеты объединены и данные не потеряны
4. Проанализирована точность ответов исполнителей
5. [Необязательно] Предложен альтернативный метод агрегации ответов и проведено сравнение с методом мнениия большинства


In [1]:
# @title import need for this
import io
import json
import pandas as pd
import requests
from urllib.parse import urlencode

# import asyncio
from collections import Counter

In [20]:
# @title download data
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
link_csv = r'https://disk.yandex.ru/d/POnzuqYYF76uuw'
link_json = r'https://disk.yandex.ru/d/9ALcR4nh0sqo4g'


def load_file(name, base, link):
    final_url = base + urlencode(dict(public_key=link))
    response = requests.get(final_url)
    if response.status_code != 200:
        print(f"Ошибка при получении загрузочной ссылки: {response.status_code}")
        print(response.text)
        return None

    download_url = response.json().get('href')
    download_response = requests.get(download_url)
    if download_response.status_code != 200:
        print(f"Ошибка при загрузке файла: {download_response.status_code}")
        print(download_response.text)
        return None

    match name:
        case 'main.csv':
            return pd.read_csv(
                io.StringIO(download_response.content.decode('utf-8')),
                sep=';'
            )
        case 'golden.json':
            try:
                json_data = download_response.json()
                return pd.json_normalize(json_data)
            except ValueError as e:
                print("Ошибка при декодировании JSON:")
                print(download_response.content.decode('utf-8'))
                raise e
        case _:
            raise ValueError("Unsupported file type")
# Загрузка файлов
data = load_file('main.csv', base_url, link_csv)
print(data)
golden = load_file('golden.json', base_url, link_json)
print(golden)

       worker    task  label
0        w851  t30685      1
1       w6991  t30008      0
2       w2596  t36316      0
3       w5507  t15145      1
4       w2982  t44785      1
...       ...     ...    ...
475531  w4660  t62250      1
475532  w6630  t46626      0
475533  w4605  t93513      1
475534  w1928  t29002      0
475535  w5375  t49052      1

[475536 rows x 3 columns]
         task  true_label
0      t30006           0
1      t33578           0
2      t22462           1
3      t52093           0
4      t26935           0
...       ...         ...
10074  t57345           1
10075  t81052           1
10076   t7189           1
10077  t80463           0
10078  t93643           0

[10079 rows x 2 columns]


# Обработка данных

In [17]:
# @title Метод мнения большинства

# Получение ответов с помощью метода большинства
# предполагает игнорирование оценки качества исполнителей
majority_votes = (
    data.groupby('task')['label'].agg(lambda x: x.mode()[0]).reset_index()
    )
majority_votes.columns = ['task', 'majority_label']
print("\nОтветы по методу большинства:")
print(majority_votes.head(10))


Ответы по методу большинства:
     task  majority_label
0      t0               1
1      t1               1
2     t10               1
3    t100               0
4   t1000               0
5  t10000               1
6  t10001               1
7  t10002               1
8  t10003               0
9  t10004               1


In [23]:
# @title Оценка качества исполнителей "в лоб"

# Здесь мы для исполнителей укажем веса благонадёжности
# опираясь исключительно на среднее по маркерным задачам
merged_data = pd.merge(data, golden, on='task', how='left')
# print("\nОбъединенные данные:")
# print(merged_data.head(), end='\n\n')

# Добавим столбец с точностью исполнителей
merged_data = merged_data.assign(
    accuracy=merged_data.groupby('worker')['true_label']
    .transform(lambda x: round(x.mean(), 2))
).sort_values(by="task")
print("\nОбъединенные данные с accuracy:")
print(merged_data.head(10), end='\n\n')


Объединенные данные с accuracy:
       worker task  label  true_label  accuracy
245223  w3585   t0      1         NaN      0.38
216734  w6254   t0      0         NaN      0.50
233299  w4316   t0      1         NaN      0.37
70606    w269   t0      1         NaN      0.59
470374   w599   t0      0         NaN      0.38
247904  w5879   t1      1         NaN      0.37
88368   w2515   t1      1         NaN      0.60
263405  w3086   t1      1         NaN      0.60
34430   w1297   t1      1         NaN      0.45
258033  w6278   t1      1         NaN      0.43



In [24]:
# @title Метод лучшей половины, с использованием весов

# Получение ответов с помощью метода большинства
# с учётом компетенции исполнителей

# Выдернем воркеров с весами
precision = merged_data[
    ['worker', 'accuracy']].drop_duplicates().reset_index(drop=True)

# Определение исполнителей для каждой задачи
task_workers = merged_data.groupby('task')['worker'].apply(list).reset_index()
task_workers.sort_values(by='task', key=lambda x: x.str[1:].astype(int), inplace=True) # type: ignore
task_workers.columns = ['task', 'workers']

def get_most_common_label(task, workers):
    # Получаем срез фрейма по задаче
    task_data = merged_data.loc[[task]]
    # print(task_data)

    # Сортируем исполнителей по их точности в порядке убывания
    sorted_workers = (
        precision[precision['worker'].isin(workers)]
        .sort_values(by='accuracy', ascending=False)
    )

    _len = len(sorted_workers)
    if _len > 2:
        half_count = (len(sorted_workers) // 2) + 1
    else:
        # если 2 или 1 учитывем все доступные
        half_count = _len

    # Берем большую половину исполнителей
    first_half_workers = sorted_workers['worker'].head(half_count).tolist()

    # print(task, first_half_workers)

    # Получаем метки от первых половины исполнителей
    labels_first_half = task_data[task_data['worker'].isin(first_half_workers)]['label']

    # Определяем наиболее частое значение label
    if not labels_first_half.empty:
        most_common_label = Counter(labels_first_half).most_common(1)
        return task, most_common_label[0][0] if most_common_label else None
    # если результата нет
    return task, None


def main():
    results = []
    for _, row in task_workers.iterrows():
        task = row['task']
        workers = row['workers']
        # print(task, workers)
        results.append(get_most_common_label(task, workers))

    results_df = pd.DataFrame(results, columns=['task', 'accuracy_label'])
    return results_df

# Добавим индекс перед долгой обработкой
merged_data.set_index('task', inplace=True)
accuracy_label = main()

merged_data = pd.merge(merged_data, majority_votes, on='task', how='left')
merged_data = pd.merge(merged_data, accuracy_label, on='task', how='left')

print("\nОбъединенные данные с accuracy_label:")
print(merged_data)


Объединенные данные с accuracy_label:
         task worker  label  true_label  accuracy  majority_label  \
0          t0  w3585      1         NaN      0.38               1   
1          t0  w6254      0         NaN      0.50               1   
2          t0  w4316      1         NaN      0.37               1   
3          t0   w269      1         NaN      0.59               1   
4          t0   w599      0         NaN      0.38               1   
...       ...    ...    ...         ...       ...             ...   
475531  t9999   w821      1         NaN      0.29               1   
475532  t9999  w4263      1         NaN      0.00               1   
475533  t9999  w5960      1         NaN      0.33               1   
475534  t9999   w381      1         NaN      0.25               1   
475535  t9999   w878      1         NaN      0.43               1   

        accuracy_label  
0                    1  
1                    1  
2                    1  
3                    1  
4      

# Сравнение подходов

In [25]:
# Выбор необходимых столбцов
selected_columns = merged_data[['worker', 'task', 'label', 'majority_label', 'accuracy_label', 'accuracy']]
# Фильтрация строк, где absolute_accuracy отличается от good_accuracy
filtered_data = selected_columns[selected_columns['majority_label'] != selected_columns['accuracy_label']]
# Печать отфильтрованных данных
print("\nСтроки, где majority_label отличается от accuracy_label:")
print(filtered_data)


Строки, где majority_label отличается от accuracy_label:
       worker   task  label  majority_label  accuracy_label  accuracy
20      w7117  t1000      1               0               1      0.38
21      w6902  t1000      1               0               1      0.50
22      w4883  t1000      0               0               1      0.29
23       w521  t1000      0               0               1      0.38
24       w661  t1000      0               0               1      0.45
...       ...    ...    ...             ...             ...       ...
475383  w5950   t997      1               1               0      0.25
475384  w5118   t997      0               1               0      0.55
475385  w6332   t997      0               1               0      0.50
475386  w1817   t997      1               1               0      0.00
475387  w6037   t997      1               1               0      0.00

[49484 rows x 6 columns]


## Вывод  
В 10 % случаев, учёт ответов исполнителей имеющих максимально возможные веса,  
среди worker-ов выполнявших задание, дают противоположный результат  
в сравнении с **"Методом мнения большенства"**.